In [8]:
# Register a signal handler that will be called when the user presses CTRL+C
signal.signal(signal.SIGINT, lambda signum, frame: youtube_dl_process.kill())

KeyboardInterrupt: 

In [5]:
print(start_time)

1670642706.6756754
